# Sales and Order Analysis  

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ecom_data_df = pd.read_parquet("../data/olist_ecom.parquet")

In [3]:
ecom_data_df.head(1)

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,...,delivery_vs_estimate,Frequency,Monetary,Recency,R_score,F_score,M_score,RFM_score_string,RFM_score_composite,RFM_segment
0,0,469634941c27cd844170935a3cf60b95,ef07ba9aa5226f77264ffa5762b2280b,81750.0,curitiba,PR,a9119eb77d6200811953803a7b6539e1,delivered,2018-03-12 13:07:03,2018-03-12 13:15:28,...,-6.229919,1,44.92,182,4,1,1,411,6,Single Buyers (Recent & Moderate/Low M)
